In [1]:
import os
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.image as image
import glob
import random

import torch
import torch.nn as nn
from torch import flatten
from torch.nn import Conv2d, ReLU, MaxPool2d
from torchvision import datasets, models, transforms
import torchvision.utils
from torchvision.io import read_image, write_png
from torch.utils.data import Dataset, DataLoader, BatchSampler, SequentialSampler, RandomSampler, Sampler

import librosa
from scipy.io.wavfile import read
from PIL import Image
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from skimage.filters import gaussian
from skimage.util import random_noise
from skimage.transform import EuclideanTransform, warp

import modeltrack.experiment as exp

%matplotlib inline

ModuleNotFoundError: No module named 'utils'

### 2.1. DataSet and Model Classes

In [3]:
class RawAudioDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform = None, start=None, end=None):
        super().__init__()
        self.img_df = dataframe
        self.img_labels = self.img_df['impairment']
        self.img_dir = img_dir
        self.transform = transform
        self.augment_type = dataframe['augment'] if 'augment' in dataframe else None
        
    def __len__(self):
        return len(self.img_df)
    
    def __getitem__(self,index):
        # create the image path and then read in the image
        img_path = os.path.join(self.img_dir, self.img_df.iloc[index]['img_path'])
        img = read_image(img_path)
        label = self.img_df.iloc[index]['impairment']

        # standardize the color to value between 0 and 1
        img = torch.div(img, 255.0)
 
        if self.transform is not None:
            img = self.transform(img)
            img = img.to(device=device)
            
        if self.augment_type is not None:
            if self.augment_type[idx] == 'gaussian':
                spec = gaussian(spec,sigma=0.5,multichannel=True)
        
        return img, label

In [5]:
class RawImageModel(nn.Module):
    def __init__(self):
        super(RawImageModel, self).__init__()

        self.vgg_16_pretrained = models.vgg16(pretrained=True)
        
        # want to freeze vgg layers
        for parameter in self.vgg_16_pretrained.parameters():
                parameter.requires_grad = False

        n_inputs = self.vgg_16_pretrained.classifier[6].in_features
        self.vgg_16_pretrained.classifier[6] = nn.Sequential(
            nn.Linear(in_features=n_inputs, out_features=256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(in_features=256, out_features=2),
            nn.LogSoftmax(dim=1)
        )
    
    def forward(self, img):
        x = self.vgg_16_pretrained(img)
        return x

### 2.2. Training and Testing Functions

In [6]:
def train_one_epoch(dataloader, model, loss_fn, optimizer):
    """
    One cycle of model training
    """
    size = len(dataloader.dataset)
    train_loss = 0
    total_correct = 0

    for batch_index, batch in enumerate(dataloader):
        batch = [r.to(device) for r in batch]
        img, labels = batch
        
        # set previous gradients to zero
        optimizer.zero_grad()

        # Compute prediction for current batch
        preds = model.forward(img.float())
        
        # compute the loss between actual and predicted values
        loss = loss_fn(preds, labels)
        train_loss += loss.item()

        # Backpropagation to compute new gradients
        loss.backward()

        # update the model parameters
        optimizer.step()

        #compute the number of correct predictions
        total_correct += (preds.argmax(1) == labels).type(torch.float).sum().item()

        # log relative information for epoch training pass
        if batch_index % 15 == 0:
            loss = loss.item()
            current = batch_index * BATCH_SIZE
            logging.info(f"training loss: {loss:>7f}  [{current}/{len(dataloader.dataset)}]")
            print(f"training loss: {loss:>7f}  [{current}/{len(dataloader.dataset)}]")

    # compute the training loss of the epoch
    avg_train_loss = train_loss / len(dataloader)
    train_accuracy = total_correct / len(dataloader.dataset)

    return avg_train_loss, train_accuracy


def test_one_epoch(dataloader, model, loss_fn):
    """
    One cycle of the model testing/validation cycle
    """
    model.eval()

    test_loss = 0
    total_correct = 0
    
    # empty list to save the model predictions
    total_preds = []

    for batch_index, batch in enumerate(dataloader):
        batch = [r.to(device) for r in batch]
        img, labels = batch

        with torch.no_grad():
            preds = model.forward(img)
            loss = loss_fn(preds, labels)
            test_loss += loss.item()
            total_correct += (preds.argmax(1) == labels).type(torch.float).sum().item()
            total_preds.append(preds)

    avg_test_loss = test_loss / len(dataloader)
    test_accuracy = total_correct / len(dataloader.dataset)       

    return avg_test_loss, test_accuracy


def save_checkpoint(model_name, model, epoch, optimizer, loss, is_best=False):
        """
        Checkpoint saver
        :param file_name: name of the checkpoint file
        :param is_best: boolean flag to indicate whether current checkpoint's accuracy is the best so far
        :return:
        """
        if is_best:
            torch.save(
                model.state_dict(),
                f"../models/{model_name}/output/model_checkpoint_best.pt.tar",
            )

        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "loss": loss
            },
            f"../models/{model_name}/checkpoint/model_checkpoint.pt.tar",
        )
        
def plot_loss(model_name, current_epoch, train_losses, test_losses, testDesc):
        plot_file_path = f"../models/{model_name}/output/loss_curve_{testDesc}.png"
        fig = plt.figure()
        plt.plot(range(1, current_epoch + 2), train_losses, color="r")
        plt.plot(range(1, current_epoch + 2), test_losses, color="b")
        plt.xlabel("epoch")
        plt.ylabel("training loss")
        fig.savefig(plot_file_path, bbox_inches="tight")
        plt.close()

## 4. Running Model: Mel-Spectrogram Images

In [7]:
data_files = glob.glob('../audio/*/*.wav')
data = pd.DataFrame({'path': data_files})
data['labels'] = data['path'].apply(lambda x: 0 if 'control' in x else 1)
data = data.sample(frac=1, random_state=2021).reset_index(drop=True)

In [3]:
experiment = exp.Experiment('my-first-experiment')
experiment
# experiment.store_params({
#     batch: 32
#     epochs: 100
#     learning_rate: 1e-4
#     device: 'cuda'
#     seed: 2021
#     test_size: 0.15
# })

NameError: name 'exp' is not defined

In [9]:
# load the data sets into their respective classes
# Note: 0.15 train_test split was used to get respective data
train, test = train_test_split(data, random_state=experiment.config.seed, test_size=experiment.config.test_size)

training_data = SpectrogramDataset(train)
test_data = SpectrogramDataset(test.reset_index())

train_loader = DataLoader(training_data, batch_size=experiment.config.batch, shuffle=True)
test_loader = DataLoader(test_data, batch_size=experiment.config.batch, shuffle=True)

print(f"Train Size: {len(train_loader.dataset)}, Test Size: {len(test_loader.dataset)}")

In [ ]:
model = RawImageModel().to(device)

class_wts = compute_class_weight("balanced", classes=np.unique(train['labels']), y=train['labels'])
weights = torch.tensor(class_wts, dtype=torch.float).to(device)
loss_fn = nn.CrossEntropyLoss(weight=weights)

optimizer = torch.optim.AdamW(model.parameters(), lr=experiment.config.learning_rate)

# set initial loss to infinite
best_test_loss = float("inf")


for current_epoch in tqdm(range(EPOCHS)):
    print(f"\nEpoch {current_epoch+1}\n-------------------------------")

    train_loss, train_acc  = train_one_epoch(train_loader, model, loss_fn, optimizer)
    
    test_loss, test_acc = test_one_epoch(test_loader, model, loss_fn)

    experiment.save_epoch_stats(train_loss, test_loss, train_acc, test_acc)

    experiment.save_model(
        model=model,
        epoch=current_epoch,
        optimizer=optimizer,
        loss=test_loss
    )

Train Size: 1876, Test Size: 83


  0%|          | 0/10 [00:00<?, ?it/s]


Epoch 1
-------------------------------


/home/ec2-user/anaconda3/envs/bert/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


training loss: 0.698319  [0/1876]


/home/ec2-user/SageMaker/notebooks/utils.py:28: RuntimeWarning: invalid value encountered in true_divide
  return (arr - arr_min) / (arr_max - arr_min)
